# Apriori Algorithm Implementation Assignment

### Objective:
You will implement the **Apriori algorithm** from scratch (i.e., without using any libraries like `mlxtend`) to find frequent itemsets and generate association rules.

### Dataset:
Use the [Online Retail Dataset](https://www.kaggle.com/datasets/vijayuv/onlineretail) from Kaggle. You can filter it for a specific country (e.g., `United Kingdom`) and time range to reduce size if needed.

---

## Step 1: Data Preprocessing

- Load the dataset
- Remove rows with missing values
- Filter out rows where `Quantity <= 0`
- Convert Data into Basket Format

👉 **Implement code below**

In [1]:
import pandas as pd

# Load the dataset
df = pd.read_csv('OnlineRetail.csv', encoding='ISO-8859-1', nrows=10)

# Preprocess as per the instructions above | We have already done in TASK 2
df.dropna(how="any", inplace=True)
df = df[df['Quantity'] > 0]

# Your Code Here
# for Basket
basket = df.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo')
basket = basket.applymap(lambda x: 1 if x > 0 else 0)

print(basket.head(10))

Description  ASSORTED COLOUR BIRD ORNAMENT  CREAM CUPID HEARTS COAT HANGER  \
InvoiceNo                                                                    
536365                                   0                               1   
536366                                   0                               0   
536367                                   1                               0   

Description  GLASS STAR FROSTED T-LIGHT HOLDER  HAND WARMER RED POLKA DOT  \
InvoiceNo                                                                   
536365                                       1                          0   
536366                                       0                          1   
536367                                       0                          0   

Description  HAND WARMER UNION JACK  KNITTED UNION FLAG HOT WATER BOTTLE  \
InvoiceNo                                                                  
536365                            0                                    

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_2100\633627972.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket = basket.applymap(lambda x: 1 if x > 0 else 0)


## Step 2: Implement Apriori Algorithm
Step-by-Step Procedure:
1. Generate Frequent 1-Itemsets
Count the frequency (support) of each individual item in the dataset.
Keep only those with support ≥ min_support.
→ Result is L1 (frequent 1-itemsets)
2. Iterative Candidate Generation (k = 2 to n)
While L(k-1) is not empty:
a. Candidate Generation

Generate candidate itemsets Ck of size k from L(k-1) using the Apriori property:
Any (k-itemset) is only frequent if all of its (k−1)-subsets are frequent.
b. Prune Candidates
Eliminate candidates that have any (k−1)-subset not in L(k-1).
c. Count Support
For each transaction, count how many times each candidate in Ck appears.
d. Generate Frequent Itemsets
Form Lk by keeping candidates from Ck that meet the min_support.
Repeat until Lk becomes empty.
Implement the following functions:
1. `get_frequent_itemsets(transactions, min_support)` - Returns frequent itemsets and their support
2. `generate_candidates(prev_frequent_itemsets, k)` - Generates candidate itemsets of length `k`
3. `calculate_support(transactions, candidates)` - Calculates the support count for each candidate

**Write reusable functions** for each part of the algorithm.

In [2]:
#---------------------------------------------------------------
# Helper: check if subset is inside a set
def is_subset(subset, superset):
    for item in subset:
        found = False
        for s in superset:
            if item == s:
                found = True
                break
        if not found:
            return False
    return True

# Helper: check if two lists are equal as sets
def are_equal_sets(a, b):
    if len(a) != len(b):
        return False
    for item in a:
        if not is_subset([item], b):
            return False
    return True

# Helper: generate all unique items
def get_unique_items(transactions):
    items = []
    for t in transactions:
        for x in t:
            already = False
            for i in items:
                if i == x:
                    already = True
                    break
            if not already:
                items.append(x)
    return [[i] for i in items]

# Helper: generate k-combinations
def generate_combinations(items, k):
    result = []
    def backtrack(start, path):
        if len(path) == k:
            result.append(path[:])
            return
        for idx in range(start, len(items)):
            path.append(items[idx])
            backtrack(idx + 1, path)
            path.pop()

    backtrack(0, [])
    return result

#---------------------------------------------------------------
def calculate_support(transactions, candidates):
    support_count = {}
    for cand in candidates:
        count = 0
        for t in transactions:
            if is_subset(cand, t):
                count += 1
        support_count[tuple(sorted(cand))] = count
    return support_count

#---------------------------------------------------------------
def generate_candidates(prev_frequent_itemsets, k):
    candidates = []
    n = len(prev_frequent_itemsets)
    for i in range(n):
        for j in range(i + 1, n):
            union = []
            for x in prev_frequent_itemsets[i]:
                if not is_subset([x], union):
                    union.append(x)
            for y in prev_frequent_itemsets[j]:
                if not is_subset([y], union):
                    union.append(y)

            # keep only if union size == k
            if len(union) == k:
                # prune step: all k-1 subsets must be frequent
                subsets = generate_combinations(union, k - 1)
                valid = True
                for sub in subsets:
                    found = False
                    for freq in prev_frequent_itemsets:
                        if are_equal_sets(sub, freq):
                            found = True
                            break
                    if not found:
                        valid = False
                        break
                if valid:
                    # avoid duplicates
                    already = False
                    for c in candidates:
                        if are_equal_sets(c, union):
                            already = True
                            break
                    if not already:
                        candidates.append(union)
    return candidates

#---------------------------------------------------------------
def get_frequent_itemsets(transactions, min_support):
    # Step 1: L1
    single_items = get_unique_items(transactions)
    support_data = {}
    all_frequent_itemsets = {}

    sup_counts = calculate_support(transactions, single_items)
    current_frequent_itemsets = []
    for item, count in sup_counts.items():
        if count >= min_support:
            current_frequent_itemsets.append(list(item))
            support_data[item] = count
            all_frequent_itemsets[item] = count

    k = 2
    while current_frequent_itemsets:
        candidates = generate_candidates(current_frequent_itemsets, k)
        sup_counts = calculate_support(transactions, candidates)

        new_frequent = []
        for item, count in sup_counts.items():
            if count >= min_support:
                new_frequent.append(list(item))
                support_data[item] = count
                all_frequent_itemsets[item] = count

        current_frequent_itemsets = new_frequent
        k += 1

    return all_frequent_itemsets
#---------------------------------------------------------------

# Example Usage
transactions = [
    ["A", "B", "C"],
    ["A", "C"],
    ["B", "C"],
    ["A", "B"],
    ["A", "B", "C"]
]

min_support = 2
print(get_frequent_itemsets(transactions, min_support))


{('A',): 4, ('B',): 4, ('C',): 4, ('A', 'B'): 3, ('A', 'C'): 3, ('B', 'C'): 3, ('A', 'B', 'C'): 2}


## Step 3: Generate Association Rules

- Use frequent itemsets to generate association rules
- For each rule `A => B`, calculate:
  - **Support**
  - **Confidence**
- Only return rules that meet a minimum confidence threshold (e.g., 0.5)

👉 **Implement rule generation function below**

In [3]:
#---------------------------------------------------------------
# Helper: check if subset is inside a set
def is_subset(subset, superset):
    for item in subset:
        found = False
        for s in superset:
            if item == s:
                found = True
                break
        if not found:
            return False
    return True

# Helper: generate combinations of list elements
def generate_combinations(items, k):
    result = []

    def backtrack(start, path):
        if len(path) == k:
            result.append(path[:])
            return
        for idx in range(start, len(items)):
            path.append(items[idx])
            backtrack(idx + 1, path)
            path.pop()

    backtrack(0, [])
    return result

#---------------------------------------------------------------
def generate_association_rules(frequent_itemsets, transactions, min_confidence=0.5):
    rules = []
    total_transactions = len(transactions)

    for itemset_tuple, support_count in frequent_itemsets.items():
        itemset = list(itemset_tuple)

        if len(itemset) >= 2:
            for i in range(1, len(itemset)):
                subsets = generate_combinations(itemset, i)
                for left in subsets:
                    right = []
                    for x in itemset:
                        if not is_subset([x], left):
                            right.append(x)

                    left_key = tuple(sorted(left))
                    right_key = tuple(sorted(right))

                    left_support_count = frequent_itemsets.get(left_key, 0)
                    right_support_count = frequent_itemsets.get(right_key, 0)

                    if left_support_count > 0 and right_support_count > 0:
                        confidence = support_count / left_support_count
                        support = support_count / total_transactions
                        lift = confidence / (right_support_count / total_transactions)

                        if confidence >= min_confidence:
                            rules.append({
                                'antecedent': left,
                                'consequent': right,
                                'support': round(support, 4),
                                'confidence': round(confidence, 4),
                                'lift': round(lift, 4)
                            })
    return rules


#---------------------------------------------------------------

# Example usage
transactions = [
    ["A", "B", "C"],
    ["A", "C"],
    ["B", "C"],
    ["A", "B"],
    ["A", "B", "C"]
]

# Get frequent itemsets first (from previous brute-force Apriori)
min_support = 2
frequent_itemsets = get_frequent_itemsets(transactions, min_support)

rules = generate_association_rules(frequent_itemsets, transactions,min_confidence=0.6)
for r in rules:
    print(r)


{'antecedent': ['A'], 'consequent': ['B'], 'support': 0.6, 'confidence': 0.75, 'lift': 0.9375}
{'antecedent': ['B'], 'consequent': ['A'], 'support': 0.6, 'confidence': 0.75, 'lift': 0.9375}
{'antecedent': ['A'], 'consequent': ['C'], 'support': 0.6, 'confidence': 0.75, 'lift': 0.9375}
{'antecedent': ['C'], 'consequent': ['A'], 'support': 0.6, 'confidence': 0.75, 'lift': 0.9375}
{'antecedent': ['B'], 'consequent': ['C'], 'support': 0.6, 'confidence': 0.75, 'lift': 0.9375}
{'antecedent': ['C'], 'consequent': ['B'], 'support': 0.6, 'confidence': 0.75, 'lift': 0.9375}
{'antecedent': ['A', 'B'], 'consequent': ['C'], 'support': 0.4, 'confidence': 0.6667, 'lift': 0.8333}
{'antecedent': ['A', 'C'], 'consequent': ['B'], 'support': 0.4, 'confidence': 0.6667, 'lift': 0.8333}
{'antecedent': ['B', 'C'], 'consequent': ['A'], 'support': 0.4, 'confidence': 0.6667, 'lift': 0.8333}


## Step 4: Output and Visualize

- Print top 10 frequent itemsets
- Print top 10 association rules (by confidence or lift)

👉 **Output results below**

In [4]:
# Print Top 10 Frequent Itemsets
print("Top 10 Frequent Itemsets:")
# Example usage
transactions = [
    ["A", "B", "C"],
    ["A", "C"],
    ["B", "C"],
    ["A", "B"],
    ["A", "B", "C"]
]
# Get frequent itemsets first
min_support = 2
frequent_itemsets = get_frequent_itemsets(transactions, min_support)

for idx, (itemset, support) in enumerate(sorted(frequent_itemsets.items(), key=lambda x: x[1], reverse=True)[:10], 1):
    print(f"{idx}. Items: {set(itemset)} | Support Count: {support}")

# Generate and sort rules by confidence
sorted_rules = sorted(rules, key=lambda x: x['confidence'], reverse=True)

# Print Top 10 Association Rules
print("\nTop 10 Association Rules (by Confidence):")
for idx, rule in enumerate(sorted_rules[:10], 1):
    print(f"{idx}. {set(rule['antecedent'])} -> {set(rule['consequent'])} "
          f"| Support: {rule['support']:.4f} "
          f"| Confidence: {rule['confidence']:.4f} "
          f"| Lift: {rule['lift']:.4f}")

Top 10 Frequent Itemsets:
1. Items: {'A'} | Support Count: 4
2. Items: {'B'} | Support Count: 4
3. Items: {'C'} | Support Count: 4
4. Items: {'A', 'B'} | Support Count: 3
5. Items: {'A', 'C'} | Support Count: 3
6. Items: {'C', 'B'} | Support Count: 3
7. Items: {'A', 'C', 'B'} | Support Count: 2

Top 10 Association Rules (by Confidence):
1. {'A'} -> {'B'} | Support: 0.6000 | Confidence: 0.7500 | Lift: 0.9375
2. {'B'} -> {'A'} | Support: 0.6000 | Confidence: 0.7500 | Lift: 0.9375
3. {'A'} -> {'C'} | Support: 0.6000 | Confidence: 0.7500 | Lift: 0.9375
4. {'C'} -> {'A'} | Support: 0.6000 | Confidence: 0.7500 | Lift: 0.9375
5. {'B'} -> {'C'} | Support: 0.6000 | Confidence: 0.7500 | Lift: 0.9375
6. {'C'} -> {'B'} | Support: 0.6000 | Confidence: 0.7500 | Lift: 0.9375
7. {'A', 'B'} -> {'C'} | Support: 0.4000 | Confidence: 0.6667 | Lift: 0.8333
8. {'A', 'C'} -> {'B'} | Support: 0.4000 | Confidence: 0.6667 | Lift: 0.8333
9. {'C', 'B'} -> {'A'} | Support: 0.4000 | Confidence: 0.6667 | Lift: 0.833

In [ ]:
# Print Top 10 Frequent Itemsets
print("Top 10 Frequent Itemsets:")

# Get frequent itemsets first
min_support = 2
frequent_itemsets = get_frequent_itemsets(basket, min_support)

for idx, (itemset, support) in enumerate(sorted(frequent_itemsets.items(), key=lambda x: x[1], reverse=True)[:10], 1):
    print(f"{idx}. Items: {set(itemset)} | Support Count: {support}")

# Generate and sort rules by confidence
sorted_rules = sorted(rules, key=lambda x: x['confidence'], reverse=True)

# Print Top 10 Association Rules
print("\nTop 10 Association Rules (by Confidence):")
for idx, rule in enumerate(sorted_rules[:10], 1):
    print(f"{idx}. {set(rule['antecedent'])} -> {set(rule['consequent'])} "
          f"| Support: {rule['support']:.4f} "
          f"| Confidence: {rule['confidence']:.4f} "
          f"| Lift: {rule['lift']:.4f}")

Top 10 Frequent Itemsets:
